# Intro to Cython

Reminder: please avoid pairs with no C experience

## Why Cython

![DevTime](whycython.png)

Outline:

* Speed up Python code using Cython
* Interact with NumPy arrays
* Release GIL and parallelize using openmp
* Wrap C/C++ code
* Using Numba to speed up Python code

## Part I: Cython

We want to integrate the function $f(x) = x^4 - 3x$.

In [ ]:
def f(x):
    y = x**4 - 3*x
    return y
    
def integrate_f(a, b, n):
    dx = (b - a) / n
    dx2 = dx / 2
    s = f(a) * dx2
    for i in range(1, n):
        s += f(a + i * dx) * dx
    s += f(b) * dx2
    return s

Now, let's time this:

In [ ]:
%timeit integrate_f(-100, 100, 100_000)

Not too bad, but this can add up. So how can we speed this up? Use Cython!

**Cython is an optimising static compiler for both the Python programming language and the extended Cython programming language** (based on Pyrex). It makes writing C extensions for Python as easy as Python itself.

Cython gives you the combined power of Python and C to let you

- write Python code that calls back and forth from and to C or C++ code natively at any point
- easily tune readable Python code into plain C performance by adding static type declarations (possibly in Python syntax)
- use combined source code level debugging to find bugs in your Python, Cython and C code
- interact efficiently with large data sets, e.g. using multi-dimensional NumPy arrays
- quickly build your applications within the large, mature and widely used CPython ecosystem
- integrate natively with existing code and data from legacy, low-level or high-performance libraries and applications

In brief, Cython code is sort of Python code with the best parts of C. Cython code needs to be compiled into C code, which is then compiled as a shared library when your packages is installed. 

Jupyter makes it easy to write and compile Cython code through `%%cython` magic. 


In [ ]:
%load_ext cython

Let's first use our plain-old Python code, and compile it to C without doing anything to it.

In [ ]:
%%cython

def f2(x):
    y = x**4 - 3*x
    return y
    
def integrate_f2(a, b, n):
    dx = (b - a) / n
    dx2 = dx / 2
    s = f2(a) * dx2
    for i in range(1, n):
        s += f2(a + i * dx) * dx
    s += f2(b) * dx2
    return s

In [ ]:
f2

In [ ]:
import sys
sys.modules[f2.__module__]

In [ ]:
!ls -l /home/zbyszek/.cache/ipython/cython/_cython_magic_8f7d8e2fe70d4b93e3ed0cd0adce1f8f.cpython-39-x86_64-linux-gnu.so

In [ ]:
%timeit integrate_f2(-100, 100, 100_000)

That's a little bit faster, which is nice since all we did was to call Cython on the exact same code. But can we do better?

An easy speed up is to annotate the type of objects you are using.

### Manual type specialization

In [ ]:
%%cython

def f3(double x):
    y = x**4 - 3*x
    return y
    
def integrate_f3(double a, double b, int n):
    dx = (b - a) / n
    dx2 = dx / 2
    s = f3(a) * dx2
    for i in range(1, n):
        s += f3(a + i * dx) * dx
    s += f3(b) * dx2
    return s

In [ ]:
%timeit integrate_f3(-100, 100, 100_000)

The final bit of "easy" Cython optimization is "declaring" the variables inside the function:

In [ ]:
%%cython

def f4(double x):
    y = x**4 - 3*x
    return y
    
def integrate_f4(double a, double b, int n):
    cdef:
        double dx = (b - a) / n
        double dx2 = dx / 2
        double s = f4(a) * dx2
        int i = 0
    for i in range(1, n):
        s += f4(a + i * dx) * dx
    s += f4(b) * dx2
    return s

In [ ]:
%timeit integrate_f4(-100, 100, 100_000)

Here's another way to declare types of elements

In [ ]:
%%cython

def f4(double x):
    y = x**4 - 3*x
    return y
    
def integrate_f4(double a, double b, int n):
    cdef double dx = (b - a) / n
    cdef double dx2 = dx / 2
    cdef double s = f4(a) * dx2
    cdef int i = 0
    for i in range(1, n):
        s += f4(a + i * dx) * dx
    s += f4(b) * dx2
    return s

3× speedup with so little effort is pretty nice. What else can we do?

Cython has a nice "-a" flag (for annotation) that can provide clues about why your code is slow.

`%%cython -a`

In [ ]:
%%cython -a

def f4(double x):
    y = x**4 - 3*x
    return y
    
def integrate_f4(double a, double b, int n):
    cdef:
        double dx = (b - a) / n
        double dx2 = dx / 2
        double s = f4(a) * dx2
        int i = 0
    for i in range(1, n):
        s += f4(a + i * dx) * dx
    s += f4(b) * dx2
    return s

## Exercise 1!

Head over to `cython-primes/exercise.ipynb`.

The goal of the exercise is to "cythonize" a simple Python function and to get familiar with the workflow.

## Function specialization

`integrate_f4()` still has a lot of yellow! How do we reduce this?



In [ ]:
%%cython -a

cdef double f(double x):
    y = x**4 - 3*x
    return y
    
def integrate_f5(double a, double b, int n):
    cdef:
        double dx = (b - a) / n
        double dx2 = dx / 2
        double s = f(a) * dx2
        int i = 0
    for i in range(1, n):
        s += f(a + i * dx) * dx
    s += f(b) * dx2
    return s

Tip: Python function calls can be expensive – in Cython doubly so because one might need to convert to and from Python objects to do the call. In our example above, the argument is assumed to be a C double both inside f() and in the call to it, yet a Python float object must be constructed around the argument in order to pass it.

Cython provides a syntax for declaring a C-style function.

https://cython.readthedocs.io/en/latest/src/tutorial/cython_tutorial.html

In [ ]:
%timeit integrate_f5(-100, 100, 10**5) 

In [ ]:
integrate_f5(-100, 100, 0)

## Optimization of arithmetic formulas


In [ ]:
%%cython -a

cdef double f(double x):
    y = (x*x*x - 3)*x
    return y
    
def integrate_f6(double a, double b, int n):
    cdef:
        double dx = (b - a) / n
        double dx2 = dx / 2
        double s = f(a) * dx2
        int i = 0
    for i in range(1, n):
        s += f(a + i * dx) * dx
    s += f(b) * dx2
    return s

In [ ]:
%timeit integrate_f6(-100, 100, 10**5) 

In [ ]:
for ifunc in [integrate_f,
              integrate_f2,
              integrate_f3,
              integrate_f4,
              integrate_f5,
              integrate_f6]:
    print(f'{ifunc.__name__} {ifunc(-100, 100, 10**5)}')    

### summary of Cython code specializations
```
  pure python:                 35 ms
  python-compatible cython:    24 ms 
  specialization of arguments: 18 ms
  full type specilization:      2.6 ms
  c-only function:              6.1 ms
  simplified expression form:   0.178 ms
```

## Exercise 2!

Head over to `cython-fibbo/exercise.ipynb`.

Watch out — this one is tricky. It shows how naive conversion to C types can lead to unexpected results.

# Break here?

## Using Cython in production code

In [ ]:
%%script true

# setup.py — don't run this in the notebook
# https://cython.readthedocs.io/en/latest/src/userguide/source_files_and_compilation.html#basic-setup-py

from setuptools import setup
from Cython.Build import cythonize

setup(
    ext_modules = cythonize("integrate_f5.pyx")
)

# run with 'python setup.py build_ext -i'

In [ ]:
%%script true

# This is Meson. Do not run in the notebook.
# https://mesonbuild.com/Cython.html

project('integrate_f5', 'cython')

py = import('python').find_installation()

py.extension_module(
    'integrate_f5',
    'integrate_f5.pyx',
    dependencies : py.dependency(),
)

# run with 'meson build-meson && meson compile -C build-meson'

# Exercise 3

Navigate to `cython-distrib/` in a terminal, follow instructions in the `README` file there.

The goal is to build `integrate_f6` using setuptools or Meson.

### When `setup.py` and when `meson.build`?

[<img src="Avena-sativa.jpg" width="100"/>](Avena-sativa.jpg)

- setuptools is standard in the Python ecosystem
- excellent integration with PyPI and other Python packages
- will download wheels from PyPI
- clumsy integration with non-Python libraries
- weak support for optional dependencies and partial rebuilds

[<img src="800px-Roundbale1.jpg" width="100"/>](800px-Roundbale1.jpg)

- Meson is arguably the best available build system for compiled code
- excellent integration with pkgconfig and other system libraries
- no integration with PyPI
- excellent support for user configuration, optional dependencies, and partial rebuilds

Thus, if setuptools is a good solution for Python projects with some Cython code, and no dependencies on system libraries. Meson is a good solution for some self-contained Python and/or Cython code, possibly alongside other non-Python libraries and executables.

# Cython architecture

[<img src="cython_architecture_small.png" width="300"/>](cython_architecture.png)

# Part II: Working with NumPy arrays in Cython

The above is a very small subset of Python. Most scientific application deal not with single values, but with arrays of data.

When dealing with arrays, we usually want to use "memoryviews".

In [ ]:
%load_ext cython

In [ ]:
%%cython -a

import cython

@cython.boundscheck(False)
@cython.wraparound(False)
def func_on_array(double [::1] arr):
    cdef size_t i, N = arr.size
    cdef double sum = 0
    for i in range(N):
        sum += arr[i]
    return sum

### A more realistic example of working with arrays

In [ ]:
# %load ../mean3filter/filter.py
import numpy as np

def mean3filter(arr):
    arr_out = np.empty_like(arr)
    for i in range(1, arr.shape[0] - 1):
        arr_out[i] = arr[i-1:i+2].sum() / 3
    arr_out[0] =  (arr[0] + arr[1]) / 2
    arr_out[-1] = (arr[-2] + arr[-1]) / 2
    return arr_out


In [ ]:
r = np.random.rand(10**6)

In [ ]:
%timeit -r 1 mean3filter(r)

In [ ]:
%%cython
import numpy as np

def mean3filter2(arr):
    arr_out = np.empty_like(arr)
    for i in range(1, arr.shape[0] - 1):
        arr_out[i] = arr[i-1:i+2].sum() / 3
    arr_out[0] =  (arr[0] + arr[1]) / 2
    arr_out[-1] = (arr[-2] + arr[-1]) / 2
    return arr_out

In [ ]:
%timeit -r 1 mean3filter2(r)

Rubbish! How do we fix this?

### Exercise: use `%%cython -a` to speed up the code

See `mean3filter/exercise.ipynb`.
The goal of the exercise is to optimize the filter function with cython.

Docs: https://cython.readthedocs.io/en/latest/src/tutorial/numpy.html

## Parallellization

**Warning:**: Dragons afoot.

In [ ]:
%%cython
import cython
from cython.parallel import prange
import numpy as np

@cython.boundscheck(False)
@cython.wraparound(False)
def mean3filter4(double[::] arr):
    cdef double[::] arr_out = np.empty_like(arr)
    cdef int i, N = arr_out.size
    with nogil:
        for i in prange(1, N-1,
                        # chunksize=(k-2) // 2, num_threads=2
                       ):
            arr_out[i] = (arr[i-1]+arr[i]+arr[i+1]) / 3
        arr_out[0] = (arr[0] + arr[1]) / 2
        arr_out[N-1] = (arr[N-1] + arr[N-2]) / 2
    return arr_out

In [ ]:
%timeit mean3filter4(r)

# Part III: wrapping external code in Cython

`cdef` defines C functions and C variables.

Cython obviously knows how to convert Python objects to C variables and back.

Wrapping an external function is similar.

From math.h:

```C
   double      expm1(double);
```

It is equivalent to `exp(x) - 1`, but accurate when $x \approx 0$.

In [ ]:
%%cython -a

cdef extern from "math.h":
    double expm1(double)
    
def native_expm1(double x):
    return expm1(x)

In [ ]:
native_expm1(0.1)

## Exercise

Head over to `cython-zstd/` and see `README`. The goal of the exercise is to wrap the compression and decompression functions for zstd.

# Part IV: Numba

In [ ]:
from numba import jit

@jit
def f(x):
    y = x**4 - 3*x
    return y
    
@jit
def integrate_f7(a, b, n):
    dx = (b - a) / n
    dx2 = dx / 2
    s = f(a) * dx2
    for i in range(1, n):
        s += f(a + i * dx) * dx
    s += f(b) * dx2
    return s

In [ ]:
%%timeit -n 1 -r 1

integrate_f7(-100, 100, 100_000)

In [ ]:
%timeit integrate_f7(-100, 100, 100_000) 

In [ ]:
integrate_f7

## Functions jitted with numba

In [ ]:
@jit
def f(a, b):
    return a + b * 2

In [ ]:
f(1, 2)
f(1.0, 2.0)
f("a", "y")

In [ ]:
f.signatures

In [ ]:
one = np.eye(5)

In [ ]:
f(one, 5)

In [ ]:
f.signatures

In [ ]:
f([1, 2], [3, 4])

In [ ]:
f.nopython_signatures

### Ahead-of-time compilation

In [ ]:
import numba

@numba.jit(numba.types.int32(numba.types.int32))
def f(x):
    y = x**4 - 3*x
    return y

In [ ]:
f(33)

In [ ]:
f(33.5)

In [ ]:
f(np.eye(3))

In [ ]:
f.signatures

When `jit()` is called with a set of types, the compilation is *eager* (happens immediately).

Doing this allows precise control over types.

It is also possible to require `nopython` mode. Numba will raise an error if this is not possible. `@numba.jit(nopython=True)` is equivalent to `@numba.njit`. It is recommended to use `njit` over `jit`.

In [ ]:
@njit
def f(...):
    ...

## Arithemetic optimization

In [ ]:
from numba import jit

@jit
def f(x):
    y = (x*x*x - 3)*x
    return y
    
@jit
def integrate_f8(a, b, n):
    dx = (b - a) / n
    dx2 = dx / 2
    s = f(a) * dx2
    for i in range(1, n):
        s += f(a + i * dx) * dx
    s += f(b) * dx2
    return s

In [ ]:
%timeit integrate_f8(-100, 100, 100_000) 

Numba is able to do loop unrolling and arithemetic optmization!

# Numba architecture

[<img src="numba_architecture.png" width="500" />](numba_architecture.png)

## Numba and array operations

### Exercise

Write a mean filter using Numba, and time it, using `-r 1 -n 1` as above. How does it compare to Cython?

In [ ]:
import numpy as np
import numba

@numba.jit
def mean3filter_nb(arr):
    arr_out = np.empty_like(arr)
    for i in range(1, arr.shape[0] - 1):
        arr_out[i] = np.sum(arr[i-1:i+2]) / 3
    arr_out[0] = (arr[0] + arr[1]) / 2
    arr_out[-1] = (arr[-1] + arr[-2]) / 2
    return arr_out

In [ ]:
r = np.random.rand(10**6)

%timeit -r 1 -n 1 mean3filter_nb(r)  # jit warmup

In [ ]:
%timeit -r 1 -n 1 mean3filter_nb(r)  # faster!

In [ ]:
# https://numba.readthedocs.io/en/stable/user/stencil.html

import numba

@numba.stencil
def _mean3filter_stencil(arr):
    return (arr[-1]+arr[0]+arr[1])/3

@numba.jit
def mean3filter_stencil(arr):
    arr_out = _mean3filter_stencil(arr)
    arr_out[0] = (arr[0] + arr[1]) / 2
    arr_out[-1] = (arr[-1] + arr[-2]) / 2
    return arr_out

In [ ]:
mean3filter_nb(r)

In [ ]:
 mean3filter_stencil(r)

In [ ]:
%timeit -r 1 -n 1 mean3filter_stencil(r)  # faster!

# Let's not forget `numpy` (and C)


In [ ]:
import numpy as np
def f(x):
    y = x**4 - 3*x
    return y

def integrate_f8(a, b, n):   
    dx = (b - a) / n
    dx2  = dx / 2
    x = np.linspace(a, b, n)
    s = f(x)
    s = s[0]*dx2 + s[1:-1].sum()*dx + s[-1]*dx2 
    
    return s

integrate_f8(-100, 100, 100_000)

In [ ]:
%timeit integrate_f8(-100, 100, 100_000)

In [ ]:
import numpy as np
def f9(x):
    y = (x*x*x - 3)*x
    return y

def integrate_f9(a, b, n):   
    dx = (b - a) / n
    dx2  = dx / 2
    x = np.linspace(a, b, n)
    s = f9(x)
    s = s[0]*dx2 + s[1:-1].sum()*dx + s[-1]*dx2 
    
    return s

integrate_f8(-100, 100, 100_000)

In [ ]:
%timeit integrate_f9(-100, 100, 100_000)

# Side demo

`c-integrate/` directory contains C code that can be compiled and used as a benchmark to compare to Cython and Numba.

### Summary of Python vs. Cython vs. Numba vs. C
```
pure python:                  33 ms
python-compatible cython:     24 ms 
specialization of arguments:  18 ms
full type specilization:      13 ms
c-only function:               6 ms
simplified expression form:    0.178 ms

numba jit:                     0.170 ms

numpy:                         7 ms
numpy simplified expression:   0.500 ms
    
plain C (-O0):                 7.3 ms
C simplified expression (-O0): 1.5 ms
C simplified expression (-O3): 0.200 ms
                               0.164 with -march=native
                               0.052 with -ffast-math                               
```                             

(For `-ffast-math` see https://gcc.gnu.org/wiki/FloatingPointMath)

# Automatic parallelization in numba

In [ ]:
def trig_ident_np(x):
    return (np.sin(x)**2 + np.cos(x)**2 +
            np.sin(x)**2 + np.cos(x)**2 +
            np.sin(x)**2 + np.cos(x)**2 +
            np.sin(x)**2 + np.cos(x)**2).sum()/4

@jit
def trig_ident_jit(x):
    s = 0    
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            s += (np.sin(x[i,j])**2 + np.cos(x[i,j])**2 +
                  np.sin(x[i,j])**2 + np.cos(x[i,j])**2 +
                  np.sin(x[i,j])**2 + np.cos(x[i,j])**2 +
                  np.sin(x[i,j])**2 + np.cos(x[i,j])**2) / 4
    return s

@jit(parallel=True)
def trig_ident_jit_parallel(x):
    return (np.sin(x)**2 + np.cos(x)**2 +
            np.sin(x)**2 + np.cos(x)**2 +
            np.sin(x)**2 + np.cos(x)**2 +
            np.sin(x)**2 + np.cos(x)**2).sum()/4

In [ ]:
x = np.random.randn(5,5)
x

In [ ]:
trig_ident_np(x)

In [ ]:
x = np.random.randn(500, 50_000)

In [ ]:
%timeit -r 1 trig_ident_np(x)

In [ ]:
%timeit trig_ident_jit(x)

In [ ]:
%timeit trig_ident_jit_parallel(x)

# Exercise

Open `numba-dot/exercise.ipynb`, see instructions therein.

## Concluding remarks

Some pros and cons about Cython and Numba:

- Cython pros:
  * very wide support
  * easy to distribute compiled code to most users
  * quite developed optimizing workflow (i.e. cython -a)
- Cython cons:
  * need to use a new language

- Numba pros:
  * quite easy to use, especially if you're starting from Cython code
  * often eye-popping, face-melting performance
  * just-in-time compilation
- Numba cons:
  * just-in-time compilation
  * requires a specific version on llvm, often not available as distro package
  * hard to optimise. If it's slow, you have to guess (though they are helpful on mailing list)
  * many parts of Python still unsupported, e.g. dicts.
  * project still young and some people are paranoid that it could disappear

# Documentation

- Exercises and repo: https://github.com/ASPP/2021-bordeaux-profiling-cython-numba
- This notebook:
- Cython:
  - https://cython.readthedocs.io/en/latest/
  - https://cython.readthedocs.io/en/latest/src/userguide/numpy_tutorial.html
- Numba:
  - http://numba.pydata.org/numba-doc/latest/index.html